# Professor's code

In [1]:
import pandas as pd
import yaml
import numpy as np
from numpy.random import seed
import doubleml as dml

from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

import random

seed(1234)

# Arianna's original code (With my own preprocessing)


### Data loading and EDA

In [2]:
import pandas as pd

# Load your dataset
rhc_df = pd.read_csv('rhc.csv')

# Assuming confounder.yml is a simple YAML file listing the confounders.

import yaml

with open('confounders.yml', 'r') as file:
    confounders = yaml.safe_load(file)


In [3]:
rhc_df.shape

(5735, 63)

In [4]:
rhc_df.describe()

,Unnamed: 0,sadmdte,dschdte,dthdte,lstctdte,cardiohx,chfhx,dementhx,psychhx,chrpulhx,...,bili1,crea1,sod1,pot1,paco21,ph1,wtkilo1,adld3p,urin1,ptid
count,5735.000000,5735.000000,5734.000000,3722.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,...,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,5735.000000,1439.000000,2707.000000,5735.000000
mean,2868.000000,11638.686312,11660.050401,11753.869156,11781.257890,0.176635,0.178030,0.098344,0.067306,0.189887,...,2.267067,2.133017,136.768963,4.066693,38.748975,7.388413,67.827817,1.182071,2192.453665,5134.006452
std,1655.696228,513.967751,513.447322,538.812330,524.094168,0.381393,0.382571,0.297805,0.250573,0.392246,...,4.801538,2.053080,7.655160,1.028353,13.183445,0.109812,29.055534,1.819057,1525.140006,2972.206379
min,1.000000,10754.000000,10757.000000,10757.000000,10756.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.099991,0.099991,101.000000,1.099854,1.000000,6.579102,0.000000,0.000000,0.000000,5.000000
25%,1434.500000,11163.500000,11184.000000,11267.000000,11316.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.799927,1.000000,132.000000,3.399902,31.000000,7.339844,56.299990,0.000000,1110.000000,2561.500000
50%,2868.000000,11759.000000,11777.000000,11831.500000,11868.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.009766,1.500000,136.000000,3.799805,37.000000,7.399998,70.000000,0.000000,1927.000000,5131.000000
75%,4301.500000,12097.000000,12120.000000,12208.000000,12244.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.399902,2.399902,142.000000,4.599609,42.000000,7.459961,83.699950,2.000000,2955.000000,7689.000000
max,5735.000000,12441.000000,12560.000000,12783.000000,12644.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,58.195312,25.097656,178.000000,11.898438,156.000000,7.769531,244.000000,7.000000,9000.000000,10278.000000


In [5]:
#display all columns
pd.set_option('display.max_columns', None) 

In [6]:
rhc_df.head()

,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,chfhx,dementhx,psychhx,chrpulhx,renalhx,liverhx,gibledhx,malighx,immunhx,transhx,amihx,age,sex,edu,surv2md1,das2d3pc,t3d30,dth30,aps1,scoma1,meanbp1,wblc1,hrt1,resp1,temp1,pafi1,alb1,hema1,bili1,crea1,sod1,pot1,paco21,ph1,swang1,wtkilo1,dnr1,ninsclas,resp,card,neuro,gastr,renal,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,COPD,NaN,Yes,11142,11151.0,NaN,11382,No,0,0,0,0,1,0,0,0,1,0,0,0,70.25098,Male,12.000000,0.640991,23.50000,30,No,46,0,41.0,22.097656,124,10.0,38.69531,68.00000,3.500000,58.000000,1.009766,1.199951,145,4.000000,40.0,7.359375,No RHC,64.69995,No,Medicare,Yes,Yes,No,No,No,No,No,No,No,No,0.0,NaN,white,Under $11k,5
1,2,MOSF w/Sepsis,NaN,No,11799,11844.0,11844.0,11844,Yes,1,1,0,0,0,0,0,0,0,1,1,0,78.17896,Female,12.000000,0.755000,14.75195,30,No,50,0,63.0,28.898438,137,38.0,38.89844,218.31250,2.599609,32.500000,0.699951,0.599976,137,3.299805,34.0,7.329102,RHC,45.69998,No,Private & Medicare,No,No,No,No,No,No,No,Yes,No,No,NaN,1437.0,white,Under $11k,7
2,3,MOSF w/Malignancy,MOSF w/Sepsis,Yes,12083,12143.0,NaN,12400,No,0,0,0,0,0,0,0,0,1,1,0,0,46.09198,Female,14.069916,0.317000,18.13672,30,No,82,0,57.0,0.049995,130,40.0,36.39844,275.50000,3.500000,21.097656,1.009766,2.599609,146,2.899902,16.0,7.359375,RHC,0.00000,No,Private,No,Yes,No,No,No,No,No,No,No,No,NaN,599.0,white,$25-$50k,9
3,4,ARF,NaN,No,11146,11183.0,11183.0,11182,Yes,0,0,0,0,0,0,0,0,0,1,0,0,75.33197,Female,9.000000,0.440979,22.92969,30,No,48,0,55.0,23.296875,58,26.0,35.79688,156.65625,3.500000,26.296875,0.399963,1.699951,117,5.799805,30.0,7.459961,No RHC,54.59998,No,Private & Medicare,Yes,No,No,No,No,No,No,No,No,No,NaN,NaN,white,$11-$25k,10
4,5,MOSF w/Sepsis,NaN,No,12035,12037.0,12037.0,12036,Yes,0,0,0,0,0,0,0,0,0,0,0,0,67.90997,Male,9.945259,0.437000,21.05078,2,Yes,72,41,65.0,29.699219,125,27.0,34.79688,478.00000,3.500000,24.000000,1.009766,3.599609,126,5.799805,17.0,7.229492,RHC,78.39996,Yes,Medicare,No,Yes,No,No,No,No,No,No,No,No,NaN,64.0,white,Under $11k,11


### Categorical treatment

In [7]:
rhc_df['ortho'] = rhc_df['ortho'].map({'No': 0, 'Yes': 1})
rhc_df['trauma'] = rhc_df['trauma'].map({'No': 0, 'Yes': 1})
rhc_df['seps'] = rhc_df['seps'].map({'No': 0, 'Yes': 1})
rhc_df['hema'] = rhc_df['hema'].map({'No': 0, 'Yes': 1})
rhc_df['meta'] = rhc_df['meta'].map({'No': 0, 'Yes': 1})
rhc_df['renal'] = rhc_df['renal'].map({'No': 0, 'Yes': 1})
rhc_df['gastr'] = rhc_df['gastr'].map({'No': 0, 'Yes': 1})
rhc_df['neuro'] = rhc_df['neuro'].map({'No': 0, 'Yes': 1})
rhc_df['card'] = rhc_df['card'].map({'No': 0, 'Yes': 1})
rhc_df['resp'] = rhc_df['resp'].map({'No': 0, 'Yes': 1})
rhc_df['dth30'] = rhc_df['dth30'].map({'No': 0, 'Yes': 1})
rhc_df['ca'] = rhc_df['ca'].map({'No': 0, 'Yes': 1})
rhc_df['sex'] = rhc_df['sex'].map({'Male': 0, 'Female': 1})
rhc_df['death'] = rhc_df['death'].map({'No': 0, 'Yes': 1})
rhc_df['swang1'] = rhc_df['swang1'].map({'No RHC': 0, 'RHC': 1})
rhc_df['dnr1'] = rhc_df['dnr1'].map({'No': 0, 'Yes': 1})


In [8]:
unique_counts_specific = rhc_df['cat1'].unique()
print(unique_counts_specific)

['COPD' 'MOSF w/Sepsis' 'MOSF w/Malignancy' 'ARF' 'CHF' 'Coma' 'Cirrhosis'
 'Lung Cancer' 'Colon Cancer']


In [9]:
unique_counts_specific = rhc_df['cat2'].unique()
print(unique_counts_specific)


[nan 'MOSF w/Sepsis' 'Coma' 'MOSF w/Malignancy' 'Lung Cancer' 'Cirrhosis'
 'Colon Cancer']


In [10]:
unique_counts_specific = rhc_df['race'].unique()
print(unique_counts_specific)


['white' 'black' 'other']


In [11]:
unique_counts_specific = rhc_df['income'].unique()
print(unique_counts_specific)

['Under $11k' '$25-$50k' '$11-$25k' '> $50k']


In [12]:
unique_counts_specific = rhc_df['ninsclas'].unique()
print(unique_counts_specific)

['Medicare' 'Private & Medicare' 'Private' 'Medicaid' 'No insurance'
 'Medicare & Medicaid']


In [13]:
from sklearn.preprocessing import LabelEncoder

categorical_columns = ['cat1', 'ninsclas', 'race', 'income']

# Initialize LabelEncoder
le = LabelEncoder()

# Apply LabelEncoder to each categorical column and update the original dataframe
for col in categorical_columns:
    rhc_df[col] = le.fit_transform(rhc_df[col])

In [14]:
rhc_df.head()

,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,chfhx,dementhx,psychhx,chrpulhx,renalhx,liverhx,gibledhx,malighx,immunhx,transhx,amihx,age,sex,edu,surv2md1,das2d3pc,t3d30,dth30,aps1,scoma1,meanbp1,wblc1,hrt1,resp1,temp1,pafi1,alb1,hema1,bili1,crea1,sod1,pot1,paco21,ph1,swang1,wtkilo1,dnr1,ninsclas,resp,card,neuro,gastr,renal,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,2,NaN,1.0,11142,11151.0,NaN,11382,0,0,0,0,0,1,0,0,0,1,0,0,0,70.25098,0,12.000000,0.640991,23.50000,30,0,46,0,41.0,22.097656,124,10.0,38.69531,68.00000,3.500000,58.000000,1.009766,1.199951,145,4.000000,40.0,7.359375,0,64.69995,0,1,1,1,0,0,0,0,0,0,0,0,0.0,NaN,2,3,5
1,2,8,NaN,0.0,11799,11844.0,11844.0,11844,1,1,1,0,0,0,0,0,0,0,1,1,0,78.17896,1,12.000000,0.755000,14.75195,30,0,50,0,63.0,28.898438,137,38.0,38.89844,218.31250,2.599609,32.500000,0.699951,0.599976,137,3.299805,34.0,7.329102,1,45.69998,0,5,0,0,0,0,0,0,0,1,0,0,NaN,1437.0,2,3,7
2,3,7,MOSF w/Sepsis,1.0,12083,12143.0,NaN,12400,0,0,0,0,0,0,0,0,0,1,1,0,0,46.09198,1,14.069916,0.317000,18.13672,30,0,82,0,57.0,0.049995,130,40.0,36.39844,275.50000,3.500000,21.097656,1.009766,2.599609,146,2.899902,16.0,7.359375,1,0.00000,0,4,0,1,0,0,0,0,0,0,0,0,NaN,599.0,2,1,9
3,4,0,NaN,0.0,11146,11183.0,11183.0,11182,1,0,0,0,0,0,0,0,0,0,1,0,0,75.33197,1,9.000000,0.440979,22.92969,30,0,48,0,55.0,23.296875,58,26.0,35.79688,156.65625,3.500000,26.296875,0.399963,1.699951,117,5.799805,30.0,7.459961,0,54.59998,0,5,1,0,0,0,0,0,0,0,0,0,NaN,NaN,2,0,10
4,5,8,NaN,0.0,12035,12037.0,12037.0,12036,1,0,0,0,0,0,0,0,0,0,0,0,0,67.90997,0,9.945259,0.437000,21.05078,2,1,72,41,65.0,29.699219,125,27.0,34.79688,478.00000,3.500000,24.000000,1.009766,3.599609,126,5.799805,17.0,7.229492,1,78.39996,1,1,0,1,0,0,0,0,0,0,0,0,NaN,64.0,2,3,11


In [15]:
rhc_df.shape

(5735, 63)

### Missing Data

In [16]:
# Identify features with missing values
features_with_missing_values = rhc_df.columns[rhc_df.isnull().any()].tolist()

# Display the count of missing values for each feature
missing_values_count = rhc_df[features_with_missing_values].isnull().sum()



print("Features with missing values:")

print(features_with_missing_values)

print("\nCount of missing values for each feature:")
print(missing_values_count)

Features with missing values:
['cat2', 'ca', 'dschdte', 'dthdte', 'adld3p', 'urin1']

Count of missing values for each feature:
cat2       4535
ca          384
dschdte       1
dthdte     2013
adld3p     4296
urin1      3028
dtype: int64


In [17]:
from sklearn.impute import KNNImputer

#Dropping these ones since they had too many Nan
columns_to_exclude = ['cat2', 'adld3p', 'Unnamed: 0']
rhc_numerical = rhc_df.drop(columns=columns_to_exclude)

imputer = KNNImputer(n_neighbors=5)
imputer.fit(rhc_numerical)
imputed_data = imputer.fit_transform(rhc_numerical)

imputed_df = pd.DataFrame(imputed_data, columns=rhc_numerical.columns)


In [18]:
# Checking of it worked
features_with_missing_values = imputed_df.columns[imputed_df.isnull().any()].tolist()
missing_values_count = imputed_df[features_with_missing_values].isnull().sum()
print("Features with missing values:")
print(features_with_missing_values)
print("\nCount of missing values for each feature:")
print(missing_values_count)

Features with missing values:
[]

Count of missing values for each feature:
Series([], dtype: float64)


In [19]:
imputed_df.columns

Index(['cat1', 'ca', 'sadmdte', 'dschdte', 'dthdte', 'lstctdte', 'death',
       'cardiohx', 'chfhx', 'dementhx', 'psychhx', 'chrpulhx', 'renalhx',
       'liverhx', 'gibledhx', 'malighx', 'immunhx', 'transhx', 'amihx', 'age',
       'sex', 'edu', 'surv2md1', 'das2d3pc', 't3d30', 'dth30', 'aps1',
       'scoma1', 'meanbp1', 'wblc1', 'hrt1', 'resp1', 'temp1', 'pafi1', 'alb1',
       'hema1', 'bili1', 'crea1', 'sod1', 'pot1', 'paco21', 'ph1', 'swang1',
       'wtkilo1', 'dnr1', 'ninsclas', 'resp', 'card', 'neuro', 'gastr',
       'renal', 'meta', 'hema', 'seps', 'trauma', 'ortho', 'urin1', 'race',
       'income', 'ptid'],
      dtype='object')

### Standardization

In [20]:
columnssc = ['cat1', 'ca', 'sadmdte', 'dschdte', 'dthdte', 'lstctdte',
       'cardiohx', 'chfhx', 'dementhx', 'psychhx', 'chrpulhx', 'renalhx',
       'liverhx', 'gibledhx', 'malighx', 'immunhx', 'transhx', 'amihx', 'age',
       'sex', 'edu', 'surv2md1', 'das2d3pc', 't3d30', 'dth30', 'aps1',
       'scoma1', 'meanbp1', 'wblc1', 'hrt1', 'resp1', 'temp1', 'pafi1', 'alb1',
       'hema1', 'bili1', 'crea1', 'sod1', 'pot1', 'paco21', 'ph1',
       'wtkilo1', 'dnr1', 'ninsclas', 'resp', 'card', 'neuro', 'gastr',
       'renal', 'meta', 'hema', 'seps', 'trauma', 'ortho', 'urin1', 'race',
       'income', 'ptid']

from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(imputed_df[columnssc])
imputed_df[columnssc]= scaler.transform(imputed_df[columnssc])

In [21]:
imputed_df.head()

,cat1,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,chfhx,dementhx,psychhx,chrpulhx,renalhx,liverhx,gibledhx,malighx,immunhx,transhx,amihx,age,sex,edu,surv2md1,das2d3pc,t3d30,dth30,aps1,scoma1,meanbp1,wblc1,hrt1,resp1,temp1,pafi1,alb1,hema1,bili1,crea1,sod1,pot1,paco21,ph1,swang1,wtkilo1,dnr1,ninsclas,resp,card,neuro,gastr,renal,meta,hema,seps,trauma,ortho,urin1,race,income,ptid
0,-0.293847,2.179981,-0.966461,-0.991685,-1.231478,-0.761872,0.0,-0.463171,-0.465391,-0.330257,-0.268632,2.065502,-0.215715,-0.274186,-0.182574,1.832458,-0.606698,-0.361241,-0.190089,0.532065,-0.892569,0.102220,0.248500,0.564843,0.635514,-0.708864,-0.434367,-0.694028,-0.986247,0.543758,0.213890,-1.285242,0.606888,-1.342175,0.519986,3.124996,-0.261877,-0.454511,1.075321,-0.064860,0.094902,-0.264461,0.0,-0.107661,-0.358768,-1.011238,1.309256,1.403554,-0.370736,-0.443325,-0.232869,-0.220105,-0.25649,-0.468161,-0.095656,-0.034958,-0.785875,0.51279,0.807824,-1.725807
1,1.504353,-0.484709,0.311941,0.358024,0.020767,0.119726,1.0,2.159028,2.148730,-0.330257,-0.268632,-0.484144,-0.215715,-0.274186,-0.182574,-0.545715,1.648267,2.768238,-0.190089,1.007350,1.120362,0.102220,0.832151,-1.080541,0.635514,-0.708864,-0.233913,-0.694028,-0.407958,1.116866,0.529107,0.703844,0.721357,-0.034462,-0.629632,0.075872,-0.326406,-0.746768,0.030183,-0.745809,-0.360254,-0.540166,1.0,-0.761637,-0.358768,1.217391,-0.763793,-0.712477,-0.370736,-0.443325,-0.232869,-0.220105,-0.25649,2.136015,-0.095656,-0.034958,-0.609784,0.51279,0.807824,-1.725134
2,1.204653,2.179981,0.864553,0.940366,0.968443,1.180696,0.0,-0.463171,-0.465391,-0.330257,-0.268632,-0.484144,-0.215715,-0.274186,-0.182574,1.832458,1.648267,-0.361241,-0.190089,-0.916275,1.120362,0.760264,-1.410131,-0.443914,0.635514,-0.708864,1.369720,-0.694028,-0.565673,-1.314220,0.359375,0.845922,-0.687451,0.463067,0.519986,-1.287546,-0.261877,0.227284,1.205963,-1.134720,-1.725722,-0.264461,1.0,-2.334623,-0.358768,0.660234,-0.763793,1.403554,-0.370736,-0.443325,-0.232869,-0.220105,-0.25649,-0.468161,-0.095656,-0.034958,-1.325420,0.51279,-0.808670,-1.724461
3,-0.893247,-0.484709,-0.958677,-0.929361,-1.269744,-1.143516,1.0,-0.463171,-0.465391,-0.330257,-0.268632,-0.484144,-0.215715,-0.274186,-0.182574,-0.545715,1.648267,-0.361241,-0.190089,0.836672,1.120362,-0.851506,-0.775436,0.457576,0.635514,-0.708864,-0.334140,-0.694028,-0.618245,0.644817,-1.386446,-0.148622,-1.026445,-0.570869,0.519986,-0.665857,-0.388889,-0.210953,-2.582661,1.685474,-0.663691,0.651603,0.0,-0.455300,-0.358768,1.217391,1.309256,-0.712477,-0.370736,-0.443325,-0.232869,-0.220105,-0.25649,-0.468161,-0.095656,-0.034958,-0.712432,0.51279,-1.616917,-1.724124
4,1.504353,-0.484709,0.771154,0.733917,0.397573,0.486104,1.0,-0.463171,-0.465391,-0.330257,-0.268632,-0.484144,-0.215715,-0.274186,-0.182574,-0.545715,-0.606698,-0.361241,-0.190089,0.391721,-0.892569,-0.551000,-0.795807,0.104179,-2.150171,1.410707,0.868585,0.660710,-0.355386,1.184349,0.238137,-0.077583,-1.589968,2.224808,0.519986,-0.940503,-0.261877,0.714400,-1.406881,1.685474,-1.649863,-1.447341,1.0,0.363892,2.787313,-1.011238,-0.763793,1.403554,-0.370736,-0.443325,-0.232869,-0.220105,-0.25649,-0.468161,-0.095656,-0.034958,-1.782300,0.51279,0.807824,-1.723788


In [22]:
# Prepare the data
Y = imputed_df['death']
D = imputed_df['swang1']
X = imputed_df[confounders]

### Randomforest no grid search (But got the same result in the regression than the doubleml)

In [83]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
import numpy as np


# Define the cross-fitting function
def cross_fit_and_calculate_residuals(model, X, y):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    residuals = np.zeros(y.shape)

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        residuals[test_index] = y.iloc[test_index] - y_pred

    return residuals

# Models
model_y = RandomForestClassifier(random_state=42)
model_d = RandomForestClassifier(random_state=42)

# Calculate residuals
Y_residuals = cross_fit_and_calculate_residuals(model_y, X, Y)
D_residuals = cross_fit_and_calculate_residuals(model_d, X, D)


In [84]:
import statsmodels.api as sm

# Add a constant to the D residuals for the intercept in OLS
D_res_with_const = sm.add_constant(D_residuals)

# Linear regression
model = sm.OLS(Y_residuals, D_res_with_const)
results = model.fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     11.40
Date:                Mon, 19 Feb 2024   Prob (F-statistic):           0.000739
Time:                        22:50:54   Log-Likelihood:                -4269.8
No. Observations:                5735   AIC:                             8544.
Df Residuals:                    5733   BIC:                             8557.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1211      0.007    -17.610      0.0

In [92]:
from sklearn.base import clone
import doubleml as dml

# Prepare DoubleML data
dml_data = dml.DoubleMLData(imputed_df, y_col='death', d_cols='swang1', x_cols=confounders)

# Setup the DoubleML model
plr = dml.DoubleMLPLR(dml_data, ml_l=clone(model_d), ml_m=clone(model_y))

# Estimate the ATE
plr.fit()

# Results
print(plr.summary)


/Users/arimichelangelo/anaconda3/lib/python3.11/site-packages/doubleml/double_ml.py:1085: UserWarning: Learner provided for ml_l is probably invalid: RandomForestClassifier(random_state=42) is (probably) no regressor.
  warnings.warn(warn_msg_prefix + f'{str(learner)} is (probably) no regressor.')


            coef   std err         t     P>|t|     2.5 %    97.5 %
swang1  0.045334  0.015715  2.884776  0.003917  0.014533  0.076135


/Users/arimichelangelo/anaconda3/lib/python3.11/site-packages/doubleml/_utils_checks.py:204: UserWarning: Propensity predictions from learner RandomForestClassifier(random_state=42) for ml_m are close to zero or one (eps=1e-12).
  warnings.warn(f'Propensity predictions from learner {str(learner)} for'


In [90]:
# DoubleMLData and DoubleMLPLR initialization
confounders_ = [c for c in imputed_df.columns if c not in ["swang1", "death"]]
data_dml = dml.DoubleMLData(
    imputed_df, y_col="death", d_cols="swang1", x_cols=confounders_
)

ml_m = GradientBoostingClassifier()
ml_l = GradientBoostingRegressor()
dml_plr_obj = dml.DoubleMLPLR(data_dml, ml_l, ml_m)

# Hyperparameter setting
learning_rate = [0.001, 0.01, 0.1]
n_estimators = [50, 100, 200]
max_depth = [1, 2, 3]

par_grids = {
    "ml_l": {
        "n_estimators": n_estimators,
        "max_depth": max_depth,
    },
    "ml_m": {
        "learning_rate": learning_rate,
        "n_estimators": n_estimators,
        "max_depth": max_depth,
    },
}

# Fine-tuning and effect estimation
dml_plr_obj.tune(param_grids=par_grids, tune_on_folds=True, search_mode="grid_search")
results = dml_plr_obj.fit()
results.summary

,coef,std err,t,P>|t|,2.5 %,97.5 %
swang1,0.022802,0.00955,2.387669,0.016956,0.004085,0.04152


### Randomforest gridsearch

In [26]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import numpy as np
from sklearn.model_selection import GridSearchCV

param_grid_classifier = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
}


def optimize_and_cross_fit(model, X, y, param_grid):
    # Initialize GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=2, scoring='accuracy', refit=True)
    
    # Fit GridSearchCV to find the best model
    grid_search.fit(X, y)
    best_model = grid_search.best_estimator_
    
    # Now perform your cross-validation with the best model
    kf = KFold(n_splits=2, shuffle=True, random_state=42)
    residuals = np.zeros(y.shape)
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        
        # Use the best model from GridSearchCV
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        residuals[test_index] = y.iloc[test_index] - y_pred
    
    return residuals, best_model


# Models 1
model_yc = RandomForestClassifier(random_state=42)
model_dc = RandomForestClassifier(random_state=42)

# Parameter grid for model optimization
param_grid_yc = param_grid_classifier # Define parameter grid for Y model
param_grid_dc = param_grid_classifier # Define parameter grid for D model

# Optimize models and calculate residuals
Y_residualsc, optimized_model_yc = optimize_and_cross_fit(model_yc, X, Y, param_grid_yc)
D_residualsc, optimized_model_dc = optimize_and_cross_fit(model_dc, X, D, param_grid_dc)



In [27]:
# Running also regressors just to check if it runs better
param_grid_regressor = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
}

# Models 1
model_yr = RandomForestRegressor(random_state=42)
model_dr = RandomForestRegressor(random_state=42)

# Parameter grid for model optimization
param_grid_yr = param_grid_regressor # Define parameter grid for Y model
param_grid_dr = param_grid_regressor # Define parameter grid for D model 

Y_residualsr, optimized_model_yr = optimize_and_cross_fit(model_yr, X, Y, param_grid_yr)
D_residualsr, optimized_model_dr = optimize_and_cross_fit(model_dr, X, D, param_grid_dr)

/Users/arimichelangelo/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/arimichelangelo/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/arimichelangelo/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/arimichelangelo/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^

In [28]:
import pickle
filename = 'rfclassificatormodel_y.sav'
pickle.dump(optimized_model_yc, open(filename, 'wb'))
filename2 = 'rfclassificatormodel_d.sav'
pickle.dump(optimized_model_dc, open(filename2, 'wb'))
filename3 = 'rfregressormodel_y.sav'
pickle.dump(optimized_model_yr, open(filename3, 'wb'))
filename4 = 'rfregressormodel_d.sav'
pickle.dump(optimized_model_dr, open(filename4, 'wb'))


In [29]:
# both classificators

import statsmodels.api as sm

# Add a constant to the D residuals for the intercept in OLS
D_res_with_constc = sm.add_constant(D_residualsc)

# Linear regression
model = sm.OLS(Y_residualsc, D_res_with_constc)
results = model.fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     16.58
Date:                Tue, 20 Feb 2024   Prob (F-statistic):           4.72e-05
Time:                        10:57:45   Log-Likelihood:                -4241.3
No. Observations:                5735   AIC:                             8487.
Df Residuals:                    5733   BIC:                             8500.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1382      0.007    -20.234      0.0

In [30]:
from sklearn.base import clone
import doubleml as dml

# Prepare DoubleML data
dml_data = dml.DoubleMLData(imputed_df, y_col='death', d_cols='swang1', x_cols=confounders)

# Setup the DoubleML model
plr = dml.DoubleMLPLR(dml_data, ml_l=clone(optimized_model_dc), ml_m=clone(optimized_model_yc))

# Estimate the ATE
plr.fit()

# Results
print(plr.summary)


/Users/arimichelangelo/anaconda3/lib/python3.11/site-packages/doubleml/double_ml.py:1085: UserWarning: Learner provided for ml_l is probably invalid: RandomForestClassifier(max_depth=30, min_samples_split=5, n_estimators=300,
                       random_state=42) is (probably) no regressor.
  warnings.warn(warn_msg_prefix + f'{str(learner)} is (probably) no regressor.')


            coef   std err         t     P>|t|    2.5 %   97.5 %
swang1  0.034085  0.016156  2.109735  0.034881  0.00242  0.06575


# With professor's preprocessing code

In [31]:
rhc = pd.read_csv("rhc.csv")
with open("confounders.yml", "r") as f:
    confounders = yaml.safe_load(f)

rhc["swang1"] = (rhc["swang1"] == "RHC").astype(int)
rhc["death"] = (rhc["death"] == "Yes").astype(int)
rhc.groupby("swang1")["death"].mean()

rhc_numerical = pd.get_dummies(rhc[['swang1', 'death'] + confounders], dtype=float, drop_first=True)

In [45]:
confounders_ = [c for c in rhc_numerical.columns if c not in ["swang1", "death"]]

# Prepare the data
Y = rhc_numerical['death']
D = rhc_numerical['swang1']
X = rhc_numerical[confounders_]

In [47]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

par_grids = {
    'learning_rate' : [0.001, 0.01, 0.1], 
    'n_estimators' : [50, 100, 200], 
    'max_depth' : [1, 2, 3]
}

def optimize_and_cross_fit(model, X, y, param_grid):
    # Initialize GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=2, scoring='accuracy', refit=True)
    
    # Fit GridSearchCV to find the best model
    grid_search.fit(X, y)
    best_model = grid_search.best_estimator_
    
    # Now perform your cross-validation with the best model
    kf = KFold(n_splits=2, shuffle=True, random_state=42)
    residuals = np.zeros(y.shape)
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        
        # Use the best model from GridSearchCV
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
        residuals[test_index] = y.iloc[test_index] - y_pred
    
    return residuals, best_model


# Models 1
model_yc = GradientBoostingClassifier(random_state=42)
model_dc = GradientBoostingClassifier(random_state=42)

# Optimize models and calculate residuals
Y_residualsc, optimized_model_yc = optimize_and_cross_fit(model_yc, X, Y, par_grids)
D_residualsc, optimized_model_dc = optimize_and_cross_fit(model_dc, X, D, par_grids)

In [48]:
# both classificators

import statsmodels.api as sm

# Add a constant to the D residuals for the intercept in OLS
D_res_with_constc = sm.add_constant(D_residualsc)

# Linear regression
model = sm.OLS(Y_residualsc, D_res_with_constc)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.108
Date:                Tue, 20 Feb 2024   Prob (F-statistic):             0.0238
Time:                        11:54:57   Log-Likelihood:                -4309.1
No. Observations:                5735   AIC:                             8622.
Df Residuals:                    5733   BIC:                             8636.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1173      0.007    -17.209      0.0

In [50]:
from sklearn.base import clone
import doubleml as dml

# Prepare DoubleML data
dml_data = dml.DoubleMLData(rhc_numerical, y_col='death', d_cols='swang1', x_cols=confounders_)

# Setup the DoubleML model
plr = dml.DoubleMLPLR(dml_data, ml_l=clone(optimized_model_dc), ml_m=clone(optimized_model_yc))

# Estimate the ATE
plr.fit()

# Results
print(plr.summary)

/Users/arimichelangelo/anaconda3/lib/python3.11/site-packages/doubleml/double_ml.py:1085: UserWarning: Learner provided for ml_l is probably invalid: GradientBoostingClassifier(n_estimators=200, random_state=42) is (probably) no regressor.
  warnings.warn(warn_msg_prefix + f'{str(learner)} is (probably) no regressor.')


            coef   std err         t     P>|t|    2.5 %    97.5 %
swang1  0.043382  0.016277  2.665246  0.007693  0.01148  0.075285


In [51]:
# DoubleMLData and DoubleMLPLR initialization
confounders_ = [c for c in rhc_numerical.columns if c not in ["swang1", "death"]]
data_dml = dml.DoubleMLData(
    rhc_numerical, y_col="death", d_cols="swang1", x_cols=confounders_
)

ml_m = GradientBoostingClassifier()
ml_l = GradientBoostingRegressor()
dml_plr_obj = dml.DoubleMLPLR(data_dml, ml_l, ml_m)

# Hyperparameter setting
learning_rate = [0.001, 0.01, 0.1]
n_estimators = [50, 100, 200]
max_depth = [1, 2, 3]

par_grids = {
    "ml_l": {
        "n_estimators": n_estimators,
        "max_depth": max_depth,
    },
    "ml_m": {
        "learning_rate": learning_rate,
        "n_estimators": n_estimators,
        "max_depth": max_depth,
    },
}

# Fine-tuning and effect estimation
dml_plr_obj.tune(param_grids=par_grids, tune_on_folds=True, search_mode="grid_search")
results = dml_plr_obj.fit()
results.summary

,coef,std err,t,P>|t|,2.5 %,97.5 %
swang1,0.042929,0.013305,3.226583,0.001253,0.016852,0.069006
